# Analyze A in Pareto

In [6]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import filter_voltages
from voltagebudget.util import read_modes
from voltagebudget.util import read_stim
from voltagebudget.util import mad
from voltagebudget.neurons import adex

# Temporal params
time_step = 1e-5
coincidence_t = 1e-3
seed_value = 42

Loading BokehJS ...

## Exp 32

In [2]:
f = 12
N = 50
t = 0.4

results = np.loadtxt("../data/exp32_d-0.2e-3_f{}.csv".format(f), delimiter=",", skiprows=1)

As = results[:, 0]

In [3]:
stim = "../data/stim1.csv"
mode = "regular"
noise = False

# Just choose
E = .21772

# Find the phase begin a osc cycle at E 
phi_E = float(-E * 2 * np.pi * f)

# -
params, w_in, bias_in, sigma = read_modes(mode)
if not noise:
    sigma = 0

stim_data = read_stim(stim)
ns = np.asarray(stim_data['ns'])
ts = np.asarray(stim_data['ts'])

spike_sets = []
for A in As:
    print(">>> Running A {}".format(A))
    
    ns_a, ts_a, _ = adex(
        N,
        t,
        ns,
        ts,
        w_in=w_in,
        bias_in=bias_in,
        f=f,
        A=A,
        phi=phi_E,
        sigma=sigma,
        budget=True,
        seed_value=seed_value,
        time_step=time_step,
        save_args=None,
        **params)
    
    spike_sets.append([A, ns_a.copy(), ts_a.copy()])

>>> Running A 3.34190952688e-12
>>> Running A 4.3196306888e-11
>>> Running A 4.44545964332e-12
>>> Running A 4.27808630091e-11
>>> Running A 3.42613139645e-11
>>> Running A 4.46234437414e-11
>>> Running A 2.25583053555e-11
>>> Running A 4.75123888109e-11
>>> Running A 7.02455460775e-12
>>> Running A 1.89811243609e-11
>>> Running A 4.78840926716e-11
>>> Running A 1.92404113634e-11
>>> Running A 3.97156491714e-12
>>> Running A 2.24583817637e-12
>>> Running A 1.04662030735e-11
>>> Running A 2.32150894946e-11
>>> Running A 3.20321102833e-11
>>> Running A 6.48094109856e-12
>>> Running A 1.89471529212e-11
>>> Running A 2.63174475166e-12
>>> Running A 2.97725493931e-11
>>> Running A 1.47851755337e-11
>>> Running A 1.26913889727e-11
>>> Running A 2.64045404667e-11
>>> Running A 4.62985805512e-12
>>> Running A 1.93773595479e-11
>>> Running A 2.06452410435e-11
>>> Running A 4.41344701182e-11
>>> Running A 1.59329822679e-11
>>> Running A 3.2418320478e-11
>>> Running A 3.87944179792e-11
>>> Runnin

In [13]:
from operator import itemgetter
spike_sets = sorted(spike_sets, key=itemgetter(0))

for A, ns_a, ts_a in spike_sets:
    p = figure(plot_width=300, plot_height=200)
    p.circle(ts_a, ns_a, color="black", alpha=0.3, legend="A: {:.2f} mad: {:.4f}".format(1e12*A, mad(ts_a)))
    p.xaxis.axis_label = 'Time (s)'
    p.yaxis.axis_label = 'N'
    p.x_range = Range1d(0, t)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    show(p)